In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy.external as sce
from sklearn import preprocessing
import pickle5 as pickle
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
import sklearn
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from utils import *


eps=1e-100


# plot umap

In [ ]:
# ad_gene_embedding.obs
ad_gene_embedding=sc.read_h5ad(f"../source_data/ad_embed.h5ad")


In [ ]:
ad_gene_embedding.obs['Atlas1'] = ad_gene_embedding.obs['STARmap']
ad_gene_embedding.obs['Atlas2'] = ad_gene_embedding.obs['Allen']
ad_gene_embedding.obs['Atlas3_2'] = ad_gene_embedding.obs['MERFISH_sa1']
ad_gene_embedding.obs['Atlas3_1'] = ad_gene_embedding.obs['MERFISH_co1']
ad_gene_embedding.obs['Atlas4'] = ad_gene_embedding.obs['Vizgen']
ad_gene_embedding.obs['Atlas5'] = ad_gene_embedding.obs['EEL']
ad_gene_embedding.obs['Atlas6'] = ad_gene_embedding.obs['Stereoseq']
ad_gene_embedding.obs['Atlas7'] = ad_gene_embedding.obs['Slideseq']

In [ ]:
for atlas_name in ['Atlas1','Atlas2','Atlas3_1','Atlas3_2','Atlas4','Atlas5','Atlas6','Atlas7']:
    ad_gene_embedding.obs[atlas_name]=ad_gene_embedding.obs[atlas_name].astype('str')
    ad_gene_embedding.obs.loc[ad_gene_embedding.obs[atlas_name]!='',atlas_name]=atlas_name

In [ ]:
# Concatenating values from columns 'A', 'B', and 'C' into a new column 'D'
ad_gene_embedding.obs['D'] = ad_gene_embedding.obs.apply(lambda row: f"{row['Atlas1']} {row['Atlas2']} {row['Atlas3_1']} {row['Atlas4']} {row['Atlas5']} {row['Atlas6']} {row['Atlas7']}  {row['Atlas3_2']}", axis=1)

# Display 

In [ ]:
ad_gene_embedding.obs['D'] = ad_gene_embedding.obs['D'].astype('str')

In [ ]:
ad_gene_embedding.obs.loc[ad_gene_embedding.obs['D']=='        ','D']='    Atlas7    '

In [ ]:
np.random.seed(50)
color_palette=np.random.rand(ad_gene_embedding.obs['D'].unique().shape[0],3)


color_dic = {}
for ind,i in enumerate(ad_gene_embedding.obs['D'].unique()):
    color_dic[i]=color_palette[ind]
    

fig,ax = plt.subplots(figsize=(10,10))
ax = sc.pl.umap(ad_gene_embedding,
                color='D',size=35,
                palette=color_dic,
                ax=ax,show=False)

# intersection genes

In [ ]:
import pandas as pd
from upsetplot import UpSet

example_data = {
    'Atlas1': list(ad_gene_embedding.obs['STARmap']=='STARmap'),
    'Atlas2': list(ad_gene_embedding.obs['Allen']=='Allen'),
    'Atlas3': list(ad_gene_embedding.obs['MERFISH_sa1']=='MERFISH_sa1'),
    'Atlas4': list(ad_gene_embedding.obs['Vizgen']=='Vizgen'),
    'Atlas5': list(ad_gene_embedding.obs['EEL']=='EEL'),
    'Atlas6': list(ad_gene_embedding.obs['Stereoseq']=='Stereoseq'),
    'Atlas7': list(ad_gene_embedding.obs['Slideseq']=='Slideseq'),
    # ... (other sets) ...
}
df = pd.DataFrame(example_data)

# Convert DataFrame to appropriate format for UpSetPlot
df_upset = df.set_index(list(df.columns))
# plt.figure(figsize=(10, 36))
# Create and show the UpSet plot
upset = UpSet(df_upset)

# Generate the plot
upset.plot()

# Get current axis
ax = plt.gca()

# Add counts above the bars
for patch in ax.patches:
    height = patch.get_height()
    if height > 0:  # if the bar is visible
        ax.text(
            patch.get_x() + patch.get_width() / 2, 
            height,  # Position at the top of the bar
            int(height),  # The count
            ha='center', va='bottom'
        )
# plt.savefig('figures_refine/geneoverlap1.png',dpi=300, transparent=True)
plt.show()
